<a href="https://colab.research.google.com/github/tanishq150802/Resume-parser-screener/blob/main/Resume_parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pdfminer.six

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 88.0 MB/s eta 0:00:00


In [2]:
resume_file_path='/content/TanishqSelot.pdf'

**Resume Text (for PDF)**

In [ ]:
from pdfminer.high_level import extract_text
print(extract_text(resume_file_path))

In [4]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

**Name** (using NLTK)

In [31]:
def extract_names(txt):
    person_names = []
 
    for sent in nltk.sent_tokenize(txt):
        for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent))):
            if hasattr(chunk, 'label') and chunk.label() == 'PERSON':
                person_names.append(
                    ' '.join(chunk_leave[0] for chunk_leave in chunk.leaves())
                )
 
    return person_names

names=extract_names(extract_text(resume_file_path))
print(names[0])

Tanishq


**Name** (using StanfordNER)

In [32]:
import nltk
from nltk.tag.stanford import StanfordNERTagger

PATH_TO_JAR='/content/stanford-ner.jar'
PATH_TO_MODEL = '/content/english.all.3class.distsim.crf.ser.gz'

tagger = StanfordNERTagger(model_filename=PATH_TO_MODEL,path_to_jar=PATH_TO_JAR, encoding='utf-8')
words = nltk.word_tokenize(extract_text(resume_file_path)) 
tagged = tagger.tag(words)
print(tagged[0][0]+' '+tagged[1][0]) #person's name is usually the first entity

Tanishq Selot


**Name** (using SpaCy)

In [ ]:
!pip install spacy

In [ ]:
!python -m spacy download en_core_web_sm

In [28]:
import spacy #considering a name to be two word entity

sentence = extract_text(resume_file_path)

nlp = spacy.load('en_core_web_sm')
doc = nlp(sentence)
print(str(doc.ents[0]).split()[0]+' '+str(doc.ents[0]).split()[1])

Tanishq Selot


**Phone No.**

In [48]:
import re

PHONE_REG = re.compile(r'[\+\(]?[1-9][0-9 .\-\(\)]{8,}[0-9]')
def extract_phone_number(resume_text):
    phone = re.findall(PHONE_REG, resume_text)
 
    if phone:
        number = ''.join(phone[0])
        return number
    return None
print(extract_phone_number(extract_text(resume_file_path)))

+91-7350440192


**Email**

In [49]:
EMAIL_REG = re.compile(r'[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+')

def extract_emails(resume_text):
    email = re.findall(EMAIL_REG, resume_text)
    if(email):
      if(len(email)==1):
        return email[0]
      elif(len(email)>1):
        return email
    return None
print(extract_emails(extract_text(resume_file_path)))

['me200003076@iiti.ac.in', 'tanishq1508@gmail.com']


**Skills** (using NLTK)

In [50]:
JOB_Desc = [
    'fastapi',
    'c++',
    'python',
    'html',
    'css',
    'node.js',
    'database',
    'react.js',
    'windows',
    'sql',
    'php'
]

In [34]:
def extract_skills(input_text):
    stop_words = set(nltk.corpus.stopwords.words('english'))
    word_tokens = nltk.tokenize.word_tokenize(input_text)
 
    # remove the stop words
    filtered_tokens = [w for w in word_tokens if w not in stop_words]
 
    # remove the punctuation
    filtered_tokens = [w for w in word_tokens if w.isalpha()]
 
    # generate n-grams (bi/tri)
    bigrams_trigrams = list(map(' '.join, nltk.everygrams(filtered_tokens, 2, 3)))
 
    #set to keep the results in.
    found_skills = set()
 
    #search for each token in our skills database
    for token in filtered_tokens:
        if token.lower() in JOB_Desc:
            found_skills.add(token)
 
    #search for each bigram and trigram in our skills database
    for ngram in bigrams_trigrams:
        if ngram.lower() in JOB_Desc:
            found_skills.add(ngram)
 
    return found_skills

print(extract_skills(extract_text(resume_file_path)))

{'SQL', 'FastAPI', 'Windows', 'Python'}


**Skills** (using stanfordNER)

In [51]:
def ner_skills(input_text):
  found=set()
  tagger = StanfordNERTagger(model_filename=PATH_TO_MODEL,path_to_jar=PATH_TO_JAR, encoding='utf-8')
  words = nltk.word_tokenize(input_text) 
  tagged = tagger.tag(words)
  for t in tagged:
    for skl in JOB_Desc:
      if(t[0].lower()==skl):
        found.add(t[0])
      # if(t[0]==skl):
      #   found.add(t[0])
  return found

print(ner_skills(extract_text(resume_file_path)))

{'Windows', 'C++', 'FastAPI', 'SQL', 'Python', 'Node.js'}


**College** (using NLTK)

In [40]:
EDUCATION = [
    'school',
    'college',
    'university',
    'academy',
    'faculty',
    'institute'
]

In [41]:
def extract_education(input_text):
    organizations = []
 
    # first get all the organization names using nltk
    for sent in nltk.sent_tokenize(input_text):
        for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent))):
            if hasattr(chunk, 'label') and chunk.label() == 'ORGANIZATION':
                organizations.append(' '.join(c[0] for c in chunk.leaves()))
    
    inst=[]
    for org in organizations:
      for ed in EDUCATION:
        if(len(org.split())>1):
          for i in org.split():
            if(i.lower()==ed):
              inst.append(org)
              break
    if(len(inst)>0):
      return inst[0]
    return None
print(extract_education(extract_text(resume_file_path)))

Mechanical Engineering Indian Institute Of Technology Indore


**College grade** (using stanfordNER)

In [53]:
def current_grade(input_text):
  grade={}
  tagger = StanfordNERTagger(model_filename=PATH_TO_MODEL,path_to_jar=PATH_TO_JAR, encoding='utf-8')
  words = nltk.word_tokenize(input_text) 
  tagged = tagger.tag(words)

  def is_number(n):
    try:
        float(n)   # Type-casting the string to `float`.
                   # If string is not a valid `float`, 
                   # it'll raise `ValueError` exception
    except ValueError:
        return False
    return True
  for t in tagged:
    if(is_number(t[0])):
      if(float(t[0])<=10.0): #cgpa is out of 10
        grade['college']=t[0]
        break
  
  return grade

print(current_grade(extract_text(resume_file_path)))

{'college': '8.35'}
